<a href="https://colab.research.google.com/github/Bharadwaj6903/Keras-Tuner/blob/main/Keras_Tuner_Regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from tensorflow import keras

In [3]:
df = pd.read_csv('california_housing_train.csv')
df1 = pd.read_csv('california_housing_test.csv')

df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


In [4]:
df.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
dtype: int64

In [5]:
df.shape

(17000, 9)

In [6]:
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000
mean,-119.562108,35.625225,28.589353,2643.664412,539.410824,1429.573941,501.221941,3.883578,207300.912353
std,2.005166,2.137340,12.586937,2179.947071,421.499452,1147.852959,384.520841,1.908157,115983.764387
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.790000,33.930000,18.000000,1462.000000,297.000000,790.000000,282.000000,2.566375,119400.000000
50%,-118.490000,34.250000,29.000000,2127.000000,434.000000,1167.000000,409.000000,3.544600,180400.000000
75%,-118.000000,37.720000,37.000000,3151.250000,648.250000,1721.000000,605.250000,4.767000,265000.000000
max,-114.310000,41.950000,52.000000,37937.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [8]:
df.dtypes

longitude             float64
latitude              float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
median_house_value    float64
dtype: object

In [4]:
X_train ,y_train = df.iloc[:,0:8],df.iloc[:,-1]
X_test ,y_test = df1.iloc[:,0:8],df1.iloc[:,-1]

In [7]:
X_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250


In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [21]:
def model_builder(hp): #hp is hyperparameter tuning technique
  model = keras.Sequential()

  units1 = hp.Int('units1', min_value=32, max_value=512, step=32)
  units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
  units3 = hp.Int('units3', min_value=32, max_value=512, step=32)

  model.add(keras.layers.Dense(units = units1,activation='relu'))
  model.add(keras.layers.Dense(units=units2 , activation='relu'))
  model.add(keras.layers.Dense(units=units3, activation='relu'))



  model.add(keras.layers.Dense(1,activation='linear')) #output layer. default act fn softmax which is indicated by below loss function used


  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])



  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.MeanSquaredLogarithmicError(

    name='mean_squared_logarithmic_error'
),
                metrics=[keras.losses.MeanSquaredLogarithmicError()])

  return model

We built the ann with 4 layers

Input layer

Hidden layer 1

Hidden layer 2

Output Layer

In [7]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.7 MB/s eta 0:00:00


In [22]:
import keras_tuner as kt

tuner = kt.Hyperband(model_builder,
                     objective='val_mean_squared_logarithmic_error',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

Reloading Tuner from my_dir/intro_to_kt/tuner0.json


In [23]:
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


In [24]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

Trial 26 Complete [00h 00m 31s]
val_mean_squared_logarithmic_error: 0.1003245860338211

Best val_mean_squared_logarithmic_error So Far: 0.1003245860338211
Total elapsed time: 00h 07m 54s


In [27]:
best_hps=tuner.get_best_hyperparameters()[0]

In [25]:
for p in ['units1','units2','units3','learning_rate']:
  print(p, tuner.get_best_hyperparameters()[0].get(p))


units1 288
units2 384
units3 288
learning_rate 0.01


In [28]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)



Epoch 1/50
425/425 [==============================] - 5s 10ms/step - loss: 2.3447 - mean_squared_logarithmic_error: 2.3447 - val_loss: 0.3487 - val_mean_squared_logarithmic_error: 0.3511
Epoch 2/50
425/425 [==============================] - 3s 6ms/step - loss: 0.1553 - mean_squared_logarithmic_error: 0.1553 - val_loss: 0.1864 - val_mean_squared_logarithmic_error: 0.1901
Epoch 3/50
425/425 [==============================] - 3s 7ms/step - loss: 0.1104 - mean_squared_logarithmic_error: 0.1104 - val_loss: 0.1319 - val_mean_squared_logarithmic_error: 0.1349
Epoch 4/50
425/425 [==============================] - 3s 6ms/step - loss: 0.0982 - mean_squared_logarithmic_error: 0.0982 - val_loss: 0.1213 - val_mean_squared_logarithmic_error: 0.1238
Epoch 5/50
425/425 [==============================] - 4s 9ms/step - loss: 0.0941 - mean_squared_logarithmic_error: 0.0941 - val_loss: 0.1153 - val_mean_squared_logarithmic_error: 0.1177
Epoch 6/50
425/425 [==============================] - 3s 6ms/step - l

In [32]:

eval_result = model.evaluate(X_test, y_test)
print("[test loss, test accuracy]:", eval_result)

94/94 [==============================] - 0s 5ms/step - loss: 0.0770 - mean_squared_logarithmic_error: 0.0769
[test loss, test accuracy]: [0.0769645944237709, 0.07692765444517136]
